# 1. Import Dependencies

In [3]:
from datetime import datetime
from io import BytesIO
from sqlalchemy import create_engine, inspect, text
from time import time

import json
import locale
import numpy as np
import pandas as pd

import requests
import time as t
from geoalchemy2 import Geometry, WKTElement
import psycopg2
from shapely import wkb, wkt

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

import matplotlib.pyplot as plt
import seaborn as sns

# 2. Database setup

Notes: put your postgresql configuration in the config.json file

In [6]:
with open('config.json') as config:
    db_config = json.load(config)

db_user = db_config['DB_USER']
db_password = db_config['DB_PASSWORD']
db_host = db_config['DB_HOST']
db_port = db_config['DB_PORT']
db_name = db_config['DB_NAME']

db_engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
gh_inspector = inspect(db_engine)

# 5. Data Augmentation

## 5.1 CER Power Stations

In [9]:
# Funtion to extract state coordinates and output the dictionary
def get_state_coordinates(df, state_col='state', wait=1):
    # Extract unique states
    state_list = df[state_col].dropna().unique().tolist()

    # Initialize dictionary to store results
    state_coords = {}

    # Nominatim API setup
    base_url = 'https://nominatim.openstreetmap.org/search'
    headers = {'User-Agent': 'COMP5339'}

    # Query OpenStreetMap for each state
    for state in state_list:
        # Skip invalid states
        if state in ['-', '', 'N/A']:
            params = {'q': 'Australia', 'format': 'json'}  # Specify Default Value
        else:
            params = {'q': state + ', Australia', 'format': 'json'}  # Specify Australian states
        
        t.sleep(wait)  # Avoid overloading the service
        
        try:
            response = requests.get(base_url, params=params, headers=headers)
            if response.status_code == 200:
                data = response.json()
                if data:
                    lat = float(data[0]['lat'])
                    lon = float(data[0]['lon'])
                    spatial_id = data[0]['osm_id']
                    state_coords[state] = (spatial_id, lat, lon)
                else:
                    state_coords[state] = (None, None, None)
            else:
                state_coords[state] = (None, None, None)
        except:
            state_coords[state] = (None, None, None)

    return state_coords

In [10]:
# Function to load existing geocode data from a .txt file
def load_geocode_data(file_path="osm_data.txt"):
    geocode_data = {}
    try:
        with open(file_path, "r") as file:
            for line in file:
                # Parse each line, format: address, state -> osm_id, lat, lon
                parts = line.strip().split("->")
                if len(parts) == 2:
                    key = parts[0]
                    values = parts[1].split(",")
                    if len(values) == 3:
                        geocode_data[key] = (values[0], float(values[1]), float(values[2]))
    except FileNotFoundError:
        pass  # Return empty dictionary if file doesn't exist
    return geocode_data

# Function to save geocode data to a .txt file
def save_geocode_data(data, file_path="osm_data.txt"):
    with open(file_path, "a") as file:
        for key, (osm_id, lat, lon) in data.items():
            file.write(f"{key}->{osm_id},{lat},{lon}\n")

In [11]:
# Function to return the latitude and longitude of the address using OpenStreetMap Nominatim API
def geocode_osm(address, state, status='Accredited', state_coords=None, wait=1, file_path="osm_data.txt"):
    # Load existing geocode data from file
    geocode_data = load_geocode_data(file_path)

    # Check if the address and state already exist in the file
    key = f"{address}, {state}"
    if key in geocode_data:
        osm_id, lat, lon = geocode_data[key]
        return osm_id, lat, lon

    # Base URL for OpenStreetMap Nominatim API
    base_url = 'https://nominatim.openstreetmap.org/search'
    headers = {'User-Agent': 'COMP5339'}

    # Only try OSM lookup if status is 'Accredited'
    if status == 'Accredited':
        params = {'q': f"{address}, {state}", 'format': 'json'}
        t.sleep(wait)
        try:
            response = requests.get(base_url, params=params, headers=headers)
            if response.status_code == 200:
                data = response.json()
                if data:  # Return first result
                    osm_id = data[0].get('osm_id')
                    lat = float(data[0].get('lat'))
                    lon = float(data[0].get('lon'))

                    # Save the new result to the file
                    geocode_data[key] = (osm_id, lat, lon)
                    save_geocode_data({key: (osm_id, lat, lon)}, file_path)

                    return osm_id, lat, lon
        except Exception as e:
            print(f"Error while fetching data from API: {e}")
            pass  # silently ignore request errors

    # Fallback to state coordinates
    if state_coords and state in state_coords:
        # Save the fallback to the file
        geocode_data[key] = state_coords[state]
        save_geocode_data({key: state_coords[state]}, file_path)
        return state_coords[state]

    # If all fails
    print(f"All fails: {key}")
    return None, None, None

In [12]:
# Query the table and load into DataFrame
table_name = 'accredited_power_stations_data'
query = f"SELECT * FROM {table_name}"

# Access the power stations table using pandas
accredited_df = pd.read_sql(query, con=db_engine)

# Add identifier
accredited_df['status'] = 'Accredited'

# Display the first few rows of the table
accredited_df.head()

,index,accreditation_code,power_station_name,state,installed_capacity,postcode,fuel_sources,accreditation_start_date,suspension_status,baseline_mwh,comment,status
0,0,SRPXQLG5,Varsity Views - Solar w SGU - QLD,QLD,0.2109,4227,Solar,2024-12-18,Unsuspended,0,None,Accredited
1,1,SRPYNS51,NNSWLHD-Byron Central Hospital - Solar - NSW,NSW,0.7203,2481,Solar,2024-12-12,Unsuspended,0,None,Accredited
2,2,SRPXQLG2,Springwood Terrace Care Community - Solar w SG...,QLD,0.1890,4127,Solar,2024-12-10,Unsuspended,0,None,Accredited
3,3,SRPYNS45,Bunnings Bennetts Green - Solar - NSW,NSW,0.2800,2290,Solar,2024-12-03,Unsuspended,0,None,Accredited
4,4,SRPYNS46,Tuggerah Home MSB1 254kW - Solar - NSW,NSW,0.2540,2259,Solar,2024-12-02,Unsuspended,0,None,Accredited


In [13]:
# Query the table and load into DataFrame
table_name = 'committed_power_stations'
query = f"SELECT * FROM {table_name}"

# Access the power stations table using pandas
comitted_df = pd.read_sql(query, con=db_engine)

# Add identifier
comitted_df['status'] = 'Comitted'

# Rename columns to ensure uniformity
comitted_df = comitted_df.rename(columns={'fuel_source': 'fuel_sources', 'project_name': 'power_station_name'})

comitted_df.head()

,index,power_station_name,state,mw_capacity,fuel_sources,committed_date,status
0,0,East Rockingham Resource Recovery Facility,WA,29.0,Biomass,2019-12-01,Comitted
1,1,Mangalore Renewable Energy Project,VIC,5.0,Solar,2021-09-01,Comitted
2,2,Orange Community Renewable Energy Park,NSW,5.0,Solar,2022-07-01,Comitted
3,3,Moorebank Logistics Park,NSW,60.0,Solar,2022-09-01,Comitted
4,4,Wangaratta Solar Farm,VIC,40.0,Solar,2023-07-01,Comitted


In [14]:
# Query the table and load into DataFrame
table_name = 'probable_power_stations'
query = f"SELECT * FROM {table_name}"

# Access the power stations table using pandas
probable_df = pd.read_sql(query, con=db_engine)

# Add identifier
probable_df['status'] = 'Probable'

# Rename columns to ensure uniformity
probable_df = probable_df.rename(columns={'fuel_source': 'fuel_sources', 'project_name': 'power_station_name'})

probable_df.head()

,index,power_station_name,state,mw_capacity,fuel_sources,status
0,0,Barnawartha Solar Farm,VIC,64.0,Solar,Probable
1,1,Barwon solar farm,VIC,250.0,Solar,Probable
2,2,Boddington Giga Energy,WA,400.0,Solar,Probable
3,3,Bulli Creek Solar project Stage 1,QLD,775.0,Solar,Probable
4,4,Bullyard Solar Farm,QLD,100.0,Solar,Probable


In [15]:
# Concatenate to make power_station df
power_station_df = pd.concat([accredited_df, comitted_df, probable_df], ignore_index=True)
# Convert the float column to string, removing .0 if it's an integer
power_station_df['postcode'] = power_station_df['postcode'].apply(lambda x: str(int(x)) if x.is_integer() else str(x))
power_station_df.head()

,index,accreditation_code,power_station_name,state,installed_capacity,postcode,fuel_sources,accreditation_start_date,suspension_status,baseline_mwh,comment,status,mw_capacity,committed_date
0,0,SRPXQLG5,Varsity Views - Solar w SGU - QLD,QLD,0.2109,4227,Solar,2024-12-18,Unsuspended,0.0,None,Accredited,NaN,NaT
1,1,SRPYNS51,NNSWLHD-Byron Central Hospital - Solar - NSW,NSW,0.7203,2481,Solar,2024-12-12,Unsuspended,0.0,None,Accredited,NaN,NaT
2,2,SRPXQLG2,Springwood Terrace Care Community - Solar w SG...,QLD,0.1890,4127,Solar,2024-12-10,Unsuspended,0.0,None,Accredited,NaN,NaT
3,3,SRPYNS45,Bunnings Bennetts Green - Solar - NSW,NSW,0.2800,2290,Solar,2024-12-03,Unsuspended,0.0,None,Accredited,NaN,NaT
4,4,SRPYNS46,Tuggerah Home MSB1 254kW - Solar - NSW,NSW,0.2540,2259,Solar,2024-12-02,Unsuspended,0.0,None,Accredited,NaN,NaT


In [16]:
# Get unique state coordinates
state_coords = get_state_coordinates(power_station_df, state_col='state', wait=1)

state_coords

{'QLD': (2316595, -22.1646782, 144.5844903),
 'NSW': (2316593, -31.8759835, 147.2869493),
 'VIC': (2316741, -36.5986096, 144.6780052),
 'SA': (2316596, -30.5343665, 135.6301212),
 'WA': (2316598, -25.2303005, 121.0187246),
 'ACT': (2354197, -35.4883502, 149.0026942),
 'TAS': (2369652, -42.035067, 146.6366887),
 'NT': (2316594, -19.8516101, 133.2303375)}

In [17]:
# Apply geocode_osm to each row and unpack results into 'Latitude' and 'Longitude'

total_rows = len(power_station_df)
for i, row in power_station_df.iterrows():
    # Print progress
    print(f"Processing {row['power_station_name']}, {row['state']}, {row['status']}... {i + 1}/{total_rows}".ljust(200), end='\r')
    
    # Get geocode data for each row
    spatial_id, latitude, longitude = geocode_osm(row['power_station_name'], row['state'], row['status'], state_coords=state_coords)
    
    # Assign the results back to the DataFrame
    power_station_df.at[i, 'spatial_id'] = spatial_id
    power_station_df.at[i, 'Latitude'] = latitude
    power_station_df.at[i, 'Longitude'] = longitude

print()

power_station_df.head()

Processing Woolsthorpe Wind Farm, VIC, Probable... 3104/3104                                                                                                                                            


,index,accreditation_code,power_station_name,state,installed_capacity,postcode,fuel_sources,accreditation_start_date,suspension_status,baseline_mwh,comment,status,mw_capacity,committed_date,spatial_id,Latitude,Longitude
0,0,SRPXQLG5,Varsity Views - Solar w SGU - QLD,QLD,0.2109,4227,Solar,2024-12-18,Unsuspended,0.0,None,Accredited,NaN,NaT,2316595,-22.164678,144.584490
1,1,SRPYNS51,NNSWLHD-Byron Central Hospital - Solar - NSW,NSW,0.7203,2481,Solar,2024-12-12,Unsuspended,0.0,None,Accredited,NaN,NaT,2316593,-31.875984,147.286949
2,2,SRPXQLG2,Springwood Terrace Care Community - Solar w SG...,QLD,0.1890,4127,Solar,2024-12-10,Unsuspended,0.0,None,Accredited,NaN,NaT,2316595,-22.164678,144.584490
3,3,SRPYNS45,Bunnings Bennetts Green - Solar - NSW,NSW,0.2800,2290,Solar,2024-12-03,Unsuspended,0.0,None,Accredited,NaN,NaT,2316593,-31.875984,147.286949
4,4,SRPYNS46,Tuggerah Home MSB1 254kW - Solar - NSW,NSW,0.2540,2259,Solar,2024-12-02,Unsuspended,0.0,None,Accredited,NaN,NaT,2316593,-31.875984,147.286949


In [18]:
# Test connection and check PostGIS status using text() for SQL
with db_engine.connect() as connection:
    result = connection.execute(text("SELECT PostGIS_full_version();"))
    postgis_version = result.fetchone()
    print(f"PostGIS Version: {postgis_version[0]}")

PostGIS Version: POSTGIS="3.6.0 0" [EXTENSION] PGSQL="140" GEOS="3.14.0-CAPI-1.20.4" PROJ="9.6.2 NETWORK_ENABLED=OFF URL_ENDPOINT=https://cdn.proj.org USER_WRITABLE_DIRECTORY=/Users/naufal/Library/Application Support/proj DATABASE_PATH=/opt/homebrew/Cellar/proj/9.6.2/share/proj/proj.db" (compiled against PROJ 9.6.2) LIBXML="2.13.8" LIBJSON="0.18" LIBPROTOBUF="1.5.2" WAGYU="0.5.0 (Internal)"


In [19]:
# Convert lat/lon to PostGIS Point
power_station_df['geom'] = power_station_df.apply(
    lambda row: WKTElement(f"POINT({row['Longitude']} {row['Latitude']})", srid=4326),
    axis=1
)

power_station_df.head()

,index,accreditation_code,power_station_name,state,installed_capacity,postcode,fuel_sources,accreditation_start_date,suspension_status,baseline_mwh,comment,status,mw_capacity,committed_date,spatial_id,Latitude,Longitude,geom
0,0,SRPXQLG5,Varsity Views - Solar w SGU - QLD,QLD,0.2109,4227,Solar,2024-12-18,Unsuspended,0.0,None,Accredited,NaN,NaT,2316595,-22.164678,144.584490,POINT(144.5844903 -22.1646782)
1,1,SRPYNS51,NNSWLHD-Byron Central Hospital - Solar - NSW,NSW,0.7203,2481,Solar,2024-12-12,Unsuspended,0.0,None,Accredited,NaN,NaT,2316593,-31.875984,147.286949,POINT(147.2869493 -31.8759835)
2,2,SRPXQLG2,Springwood Terrace Care Community - Solar w SG...,QLD,0.1890,4127,Solar,2024-12-10,Unsuspended,0.0,None,Accredited,NaN,NaT,2316595,-22.164678,144.584490,POINT(144.5844903 -22.1646782)
3,3,SRPYNS45,Bunnings Bennetts Green - Solar - NSW,NSW,0.2800,2290,Solar,2024-12-03,Unsuspended,0.0,None,Accredited,NaN,NaT,2316593,-31.875984,147.286949,POINT(147.2869493 -31.8759835)
4,4,SRPYNS46,Tuggerah Home MSB1 254kW - Solar - NSW,NSW,0.2540,2259,Solar,2024-12-02,Unsuspended,0.0,None,Accredited,NaN,NaT,2316593,-31.875984,147.286949,POINT(147.2869493 -31.8759835)


In [20]:
table_name = 'spatial_power_stations_data'

# store data to database
power_station_df.to_sql(name=table_name, con=db_engine, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid=4326)})

104

In [21]:
# Check Postgis
query = text("SELECT power_station_name, ST_AsText(geom) AS geom_wkt FROM spatial_power_stations_data LIMIT 5;")
df_check = pd.read_sql(query, con=db_engine)
print(df_check)

                                  power_station_name  \
0                  Varsity Views - Solar w SGU - QLD   
1       NNSWLHD-Byron Central Hospital - Solar - NSW   
2  Springwood Terrace Care Community - Solar w SG...   
3              Bunnings Bennetts Green - Solar - NSW   
4             Tuggerah Home MSB1 254kW - Solar - NSW   

                         geom_wkt  
0  POINT(144.5844903 -22.1646782)  
1  POINT(147.2869493 -31.8759835)  
2  POINT(144.5844903 -22.1646782)  
3  POINT(147.2869493 -31.8759835)  
4  POINT(147.2869493 -31.8759835)  


## 5.2 NGER Data

In [23]:
# Query the table and load into DataFrame
table_name = 'greenhouse_and_energy'
query = f"SELECT * FROM {table_name}"

# Access the power stations table using pandas
nger_df = pd.read_sql(query, con=db_engine)

In [24]:
unique_facility = nger_df[['facility_name', 'state']].drop_duplicates()

unique_facility.head()

,facility_name,state
0,Cathedral Rocks Wind Farm,SA
1,Gunning Wind Farm,NSW
2,Mortlake South Wind Farm,VIC
3,Mt Gellibrand Wind Farm,VIC
4,Waubra Wind Farm,VIC


In [25]:
# Get unique state coordinates
state_coords = get_state_coordinates(unique_facility, state_col='state', wait=1)

state_coords

{'SA': (2316596, -30.5343665, 135.6301212),
 'NSW': (2316593, -31.8759835, 147.2869493),
 'VIC': (2316741, -36.5986096, 144.6780052),
 'QLD': (2316595, -22.1646782, 144.5844903),
 'WA': (2316598, -25.2303005, 121.0187246),
 'ACT': (2354197, -35.4883502, 149.0026942),
 'NT': (2316594, -19.8516101, 133.2303375),
 'TAS': (2369652, -42.035067, 146.6366887)}

In [26]:
# Apply geocode_osm to each row and unpack results into 'Latitude' and 'Longitude'

total_rows = len(unique_facility)
for j, row in unique_facility.iterrows():
    # Print progress
    print(f"Processing row {j + 1}/{total_rows}".ljust(2000), end='\r')
    
    # Get geocode data for each row
    spatial_id, latitude, longitude = geocode_osm(row['facility_name'], row['state'], state_coords=state_coords)
    
    # Assign the results back to the DataFrame
    unique_facility.at[j, 'spatial_id'] = spatial_id
    unique_facility.at[j, 'Latitude'] = latitude
    unique_facility.at[j, 'Longitude'] = longitude


print()

unique_facility.head()

Processing row 4880/858                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

,facility_name,state,spatial_id,Latitude,Longitude
0,Cathedral Rocks Wind Farm,SA,2316596,-30.534367,135.630121
1,Gunning Wind Farm,NSW,2316593,-31.875984,147.286949
2,Mortlake South Wind Farm,VIC,2316741,-36.598610,144.678005
3,Mt Gellibrand Wind Farm,VIC,2316741,-36.598610,144.678005
4,Waubra Wind Farm,VIC,203440016,-37.352857,143.631327


In [27]:
# Left join back to the full NGER data to get the spatial columns
nger_df = nger_df.merge(
    unique_facility[['facility_name', 'state', 'spatial_id', 'Latitude', 'Longitude']],
    on=['facility_name', 'state'],
    how='left'
)

In [28]:
# Convert lat/lon to PostGIS Point
nger_df['geom'] = nger_df.apply(
    lambda row: WKTElement(f"POINT({row['Longitude']} {row['Latitude']})", srid=4326),
    axis=1
)

nger_df.head()

,index,reporting_entity,facility_name,type,state,electricity_production_gj,electricity_production_mwh,total_scope_1_emissions_t_co2_e,total_scope_2_emissions_t_co2_e,total_emissions_t_co2_e,emission_intensity_t_co2_emwh,grid_connected,grid,primary_fuel,important_notes,time_period_start,time_period_stop,dataset_id,spatial_id,Latitude,Longitude,geom
0,0,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,F,SA,481948.0,133874.0,57.0,127.0,184,0.0,On,NEM,Wind,-,2023-01-07,2024-06-30,ID0243,2316596,-30.534367,135.630121,POINT(135.6301212 -30.5343665)
1,1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,NSW,491409.0,136502.0,50.0,218.0,268,0.0,On,NEM,Wind,-,2023-01-07,2024-06-30,ID0243,2316593,-31.875984,147.286949,POINT(147.2869493 -31.8759835)
2,2,ACCIONA ENERGY OCEANIA PTY LTD,Mortlake South Wind Farm,F,VIC,1019352.0,283153.0,202.0,1128.0,1330,0.0,On,NEM,Wind,-,2023-01-07,2024-06-30,ID0243,2316741,-36.598610,144.678005,POINT(144.6780052 -36.5986096)
3,3,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,F,VIC,1025451.0,284847.0,99.0,1273.0,1372,0.0,On,NEM,Wind,-,2023-01-07,2024-06-30,ID0243,2316741,-36.598610,144.678005,POINT(144.6780052 -36.5986096)
4,4,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,VIC,1954964.0,543046.0,186.0,1114.0,1300,0.0,On,NEM,Wind,-,2023-01-07,2024-06-30,ID0243,203440016,-37.352857,143.631327,POINT(143.6313265 -37.3528575)


In [29]:
table_name = 'spatial_greenhouse_and_energy'

# store data to database
nger_df.to_sql(name=table_name, con=db_engine, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid=4326)})

880

# 6. Data Transformation and Storage

## 6.1 Data Transformation

In [32]:
# Transform location information for loading into the dim_location table 

# Start with the states from NGER and CER data
# Query both tables and combine them into one DataFrame
query = """
SELECT state FROM spatial_greenhouse_and_energy
UNION
SELECT state FROM spatial_power_stations_data
"""

# Access and load into a single DataFrame
states_df = pd.read_sql(query, con=db_engine)

# Add a row with state = 'N/A'
states_df = pd.concat(
    [states_df, pd.DataFrame([{"state": "N/A"}])],
    ignore_index=True
)

# Get unique state coordinates from the consolidated state list
state_coords = get_state_coordinates(states_df, state_col='state', wait=1)

# Convert the dictionary into a DataFrame with state_code, spatial_id, latitude, and longitude
states_df = pd.DataFrame.from_dict(state_coords, orient='index', columns=['spatial_id', 'Latitude', 'Longitude'])

# Add state_code and reorganize columns
states_df['state_code'] = states_df.index
states_df = states_df[['state_code', 'spatial_id', 'Latitude', 'Longitude']]

# Filter out invalid state_code values and reset index
# states_df = states_df[~states_df['state_code'].isin(['-', '', 'N/A'])].reset_index(drop=True)

# Convert the float column to string, removing .0 if it's an integer
states_df['spatial_id'] = states_df['spatial_id'].apply(lambda x: str(int(x)) if x.is_integer() else str(x))

# Convert lat/lon to PostGIS Point
states_df['geom'] = states_df.apply(
    lambda row: WKTElement(f"POINT({row['Longitude']} {row['Latitude']})", srid=4326),
    axis=1
)

states_df

,state_code,spatial_id,Latitude,Longitude,geom
VIC,VIC,2316741,-36.598610,144.678005,POINT(144.6780052 -36.5986096)
QLD,QLD,2316595,-22.164678,144.584490,POINT(144.5844903 -22.1646782)
WA,WA,2316598,-25.230300,121.018725,POINT(121.0187246 -25.2303005)
TAS,TAS,2369652,-42.035067,146.636689,POINT(146.6366887 -42.035067)
ACT,ACT,2354197,-35.488350,149.002694,POINT(149.0026942 -35.4883502)
SA,SA,2316596,-30.534367,135.630121,POINT(135.6301212 -30.5343665)
NSW,NSW,2316593,-31.875984,147.286949,POINT(147.2869493 -31.8759835)
NT,NT,2316594,-19.851610,133.230337,POINT(133.2303375 -19.8516101)
N/A,N/A,80500,-24.776109,134.755000,POINT(134.755 -24.7761086)


In [33]:
# Now get the location information of each facility and power stations
# Query both tables and combine them into one DataFrame
query = f"""
    SELECT "spatial_id", "state" as state_code, "Latitude", "Longitude", "geom", "postcode"
    FROM spatial_power_stations_data
    UNION ALL
    SELECT "spatial_id", "state" as state_code, "Latitude", "Longitude", "geom", NULL as postcode
    FROM spatial_greenhouse_and_energy
"""

# Access the tables and load into a single DataFrame
power_stations_locations_df = pd.read_sql(query, con=db_engine)

# Convert the 'geom' column from WKB hex to geometry (apply once after concatenation)
power_stations_locations_df['geom'] = power_stations_locations_df['geom'].apply(lambda x: wkb.loads(bytes.fromhex(x)))

# Filter out invalid spatial_id values and reset index
# Filter out NaN spatial_id
power_stations_locations_df = power_stations_locations_df[
    power_stations_locations_df['spatial_id'].notna()
].reset_index(drop=True)

# Convert the float column to string, removing .0 if it's an integer
power_stations_locations_df['spatial_id'] = power_stations_locations_df['spatial_id'].apply(
    lambda x: str(int(float(x))) if isinstance(x, (float, int)) and float(x).is_integer() else str(x)
)

# Display the first few rows for verification
power_stations_locations_df.head()

,spatial_id,state_code,Latitude,Longitude,geom,postcode
0,2316595,QLD,-22.164678,144.584490,POINT (144.5844903 -22.1646782),4227
1,2316593,NSW,-31.875984,147.286949,POINT (147.2869493 -31.8759835),2481
2,2316595,QLD,-22.164678,144.584490,POINT (144.5844903 -22.1646782),4127
3,2316593,NSW,-31.875984,147.286949,POINT (147.2869493 -31.8759835),2290
4,2316593,NSW,-31.875984,147.286949,POINT (147.2869493 -31.8759835),2259


In [34]:
# Concatenate to make locations df
location_df = pd.concat([power_stations_locations_df, states_df], ignore_index=True)

# convert loc id to int
location_df['spatial_id'] = location_df['spatial_id'].apply(
    lambda x: int(float(x)) if pd.notnull(x) else None
).astype('Int64')   # Use Int64 to allow nulls

# Group by 'spatial_id' and take the first non-null value for each column in the group
location_df_grouped = location_df.groupby('spatial_id', as_index=False).first()

# Rename the columns to the desired names
location_df_grouped = location_df_grouped.rename(columns={
    'spatial_id': 'location_id',      # Rename 'spatial_id' to 'location_id'
    'state_code': 'state_code',       # 'state_code' remains the same (or change it if needed)
    'Latitude': 'latitude',           # 'Latitude' to 'latitude'
    'Longitude': 'longitude',         # 'Longitude' to 'longitude'
    'geom': 'geometry'                # 'geom' to 'geometry'
})

# Reorder the columns as needed
dim_location = location_df_grouped[['location_id', 'latitude', 'longitude', 'geometry', 'postcode', 'state_code']]

# Here we're converting the 'geometry' to a valid WKT representation
dim_location['geometry'] = dim_location.apply(
    lambda row: f"POINT({row['longitude']} {row['latitude']})"
    if pd.notnull(row['longitude']) and pd.notnull(row['latitude'])
    else None,
    axis=1
)

# Final dim_location table
dim_location.head()

,location_id,latitude,longitude,geometry,postcode,state_code
0,80500,-24.776109,134.755000,POINT(134.755 -24.7761086),None,N/A
1,2316593,-31.875984,147.286949,POINT(147.2869493 -31.8759835),2481,NSW
2,2316594,-19.851610,133.230337,POINT(133.2303375 -19.8516101),822,NT
3,2316595,-22.164678,144.584490,POINT(144.5844903 -22.1646782),4227,QLD
4,2316596,-30.534367,135.630121,POINT(135.6301212 -30.5343665),5094,SA


In [35]:
# Use the tansformed states_df to make the dim_states table
# Create a mapping of state_code to state_name
state_name_mapping = {
    'NSW': 'New South Wales',
    'VIC': 'Victoria',
    'TAS': 'Tasmania',
    'ACT': 'Australian Capital Territory',
    'QLD': 'Queensland',
    'WA': 'Western Australia',
    'SA': 'South Australia',
    'NT': 'Northern Territory',
    'N/A': 'Australia'
}

temp_dim_states_df = states_df

# Add state_name column based on the mapping
temp_dim_states_df['state_name'] = temp_dim_states_df['state_code'].map(state_name_mapping)

# Reorganize columns to match desired format
dim_states = temp_dim_states_df[['state_code', 'state_name']]

# Final dim_states data
dim_states

,state_code,state_name
VIC,VIC,Victoria
QLD,QLD,Queensland
WA,WA,Western Australia
TAS,TAS,Tasmania
ACT,ACT,Australian Capital Territory
SA,SA,South Australia
NSW,NSW,New South Wales
NT,NT,Northern Territory
N/A,N/A,Australia


In [36]:
# Transform fuel information from NGER and CER for loading into the dim_fuel table 

# Query both tables and combine them into one DataFrame
query = f"""
    SELECT "fuel_sources" as fuel_name
    FROM spatial_power_stations_data
    UNION ALL
    SELECT "primary_fuel" as fuel_name
    FROM spatial_greenhouse_and_energy
"""

# Access the tables and load into a single DataFrame
fuel_df = pd.read_sql(query, con=db_engine)

# Group by 'fuel_name' and take the first non-null value for each column in the group
fuel_df = fuel_df.groupby('fuel_name', as_index=False).first()

# Filter out invalid fuel_name values and reset index
fuel_df = fuel_df[~fuel_df['fuel_name'].isin(['-', '', 'N/A'])].reset_index(drop=True)

# Assign a unique fuel_id to each distinct fuel_name
fuel_df['fuel_id'] = range(1, len(fuel_df) + 1)

# Reorganize columns to match desired format
dim_fuel = fuel_df[['fuel_id', 'fuel_name']]

# Final dim_fuel table
dim_fuel.head()

,fuel_id,fuel_name
0,1,Bagasse
1,2,Battery
2,3,Biofuel
3,4,Biogas
4,5,Biomass


In [37]:
# Transform date information from NGER and CER for loading into the dim_time table 
# Query both tables and combine them into one DataFrame
query = """
    SELECT "accreditation_start_date" AS date_1, "committed_date" AS date_2
    FROM spatial_power_stations_data
    UNION ALL
    SELECT "time_period_start" AS date_1, "time_period_stop" AS date_2
    FROM spatial_greenhouse_and_energy
"""

# Access the tables and load into a single DataFrame
time_df = pd.read_sql(query, con=db_engine)

# Melt the DataFrame to create a single column for dates
time_df_melted = time_df.melt(value_name='date', var_name='date_type')

# Drop the date_type column if not needed
time_df_melted = time_df_melted.drop(columns=['date_type'])

# Group by 'date' to ensure no duplicates (keep first value for each date)
time_df_melted = time_df_melted.groupby('date', as_index=False).first()

# Ensure 'date' is in datetime format
time_df_melted['date'] = pd.to_datetime(time_df_melted['date'])

# Drop rows with NaN/NaT values in the 'date' column
time_df_melted = time_df_melted.dropna(subset=['date'])

# Extract the required components
time_df_melted['day'] = time_df_melted['date'].dt.day
time_df_melted['month'] = time_df_melted['date'].dt.month
time_df_melted['month_name'] = time_df_melted['date'].dt.month_name()
time_df_melted['quarter'] = time_df_melted['date'].dt.quarter
time_df_melted['year'] = time_df_melted['date'].dt.year
time_df_melted['week_of_year'] = time_df_melted['date'].dt.isocalendar().week
time_df_melted['day_of_week'] = time_df_melted['date'].dt.weekday + 1  # Monday=1, Sunday=7
time_df_melted['day_name'] = time_df_melted['date'].dt.day_name()
time_df_melted['is_weekend'] = time_df_melted['day_of_week'].isin([6, 7])  # Saturday=6, Sunday=7

# Create time_id (usually in YYYYMMDD format)
time_df_melted['time_id'] = time_df_melted['date'].dt.strftime('%Y%m%d').astype(int)

# Reorder columns to match desired structure for dim_time
dim_time = time_df_melted[['time_id', 'date', 'day', 'month', 'month_name', 'quarter', 'year', 'week_of_year', 'day_of_week', 'day_name', 'is_weekend']]

# Final dim_time table
dim_time.head()

,time_id,date,day,month,month_name,quarter,year,week_of_year,day_of_week,day_name,is_weekend
0,20010401,2001-04-01,1,4,April,2,2001,13,7,Sunday,True
1,20010403,2001-04-03,3,4,April,2,2001,14,2,Tuesday,False
2,20010404,2001-04-04,4,4,April,2,2001,14,3,Wednesday,False
3,20010417,2001-04-17,17,4,April,2,2001,16,2,Tuesday,False
4,20010420,2001-04-20,20,4,April,2,2001,16,5,Friday,False


In [38]:
# Transform power station data from CER for loading into the dim_cer_station table 
# Query the table and set it into a DataFrame
query = """
    SELECT *
    FROM spatial_power_stations_data
"""

# Access the tables and load into a single DataFrame
cer_df = pd.read_sql(query, con=db_engine)

# Filter out rows with NaN in 'power_station_name' to avoid issues with composite_key
cer_df = cer_df[cer_df['power_station_name'].notna()]

# Create a unique combination of 'accreditation_code' and 'power_station_name'
cer_df['composite_key'] = cer_df['power_station_name'].str.cat(
    cer_df['accreditation_code'].fillna('A station_'), sep="_"
)

# Get unique combinations of the composite key
unique_combinations = cer_df['composite_key'].drop_duplicates()

# Assign sequential IDs to each unique combination
location_id_mapping = pd.Series(range(1, len(unique_combinations) + 1), index=unique_combinations).to_dict()

# Map the location_id back to the original DataFrame based on the composite key
cer_df['cer_station_code'] = cer_df['composite_key'].map(location_id_mapping)

# Create time_id (in YYYYMMDD format) for accreditation_start_date and comitted_id, leaving NaN as null
cer_df['accreditation_start_date_id'] = cer_df['accreditation_start_date'].apply(
    lambda x: x.strftime('%Y%m%d') if pd.notna(x) else np.nan
).astype('Int64')

cer_df['committed_date_id'] = cer_df['committed_date'].apply(
    lambda x: x.strftime('%Y%m%d') if pd.notna(x) else np.nan
).astype('Int64')

# Now, merge cer_df with dim_fuel to get the fuel_id based on fuel_sources (fuel_name)
cer_df = cer_df.merge(dim_fuel, how='left', left_on='fuel_sources', right_on='fuel_name')

# Drop the 'fuel_name' and 'fuel_sources' column as it's no longer needed
cer_df = cer_df.drop(columns=['fuel_name', 'fuel_sources'])

# Rename the columns
cer_df = cer_df.rename(columns={
    'spatial_id': 'location_id', 
    'power_station_name': 'station_name'
})

cer_df['location_id'] = cer_df['location_id'].apply(
    lambda x: int(float(x)) if pd.notnull(x) else None
).astype('Int64')   # Use Int64 to allow nulls

# Reorder columns to match desired structure for dim_time
dim_cer_station = cer_df[['cer_station_code', 'accreditation_code', 'station_name', 'status', 'accreditation_start_date_id', 'baseline_mwh', 'installed_capacity', 'committed_date_id', 'suspension_status', 'fuel_id', 'location_id']]

dim_cer_station.head()

,cer_station_code,accreditation_code,station_name,status,accreditation_start_date_id,baseline_mwh,installed_capacity,committed_date_id,suspension_status,fuel_id,location_id
0,1,SRPXQLG5,Varsity Views - Solar w SGU - QLD,Accredited,20241218,0.0,0.2109,<NA>,Unsuspended,19,2316595
1,2,SRPYNS51,NNSWLHD-Byron Central Hospital - Solar - NSW,Accredited,20241212,0.0,0.7203,<NA>,Unsuspended,19,2316593
2,3,SRPXQLG2,Springwood Terrace Care Community - Solar w SG...,Accredited,20241210,0.0,0.1890,<NA>,Unsuspended,19,2316595
3,4,SRPYNS45,Bunnings Bennetts Green - Solar - NSW,Accredited,20241203,0.0,0.2800,<NA>,Unsuspended,19,2316593
4,5,SRPYNS46,Tuggerah Home MSB1 254kW - Solar - NSW,Accredited,20241202,0.0,0.2540,<NA>,Unsuspended,19,2316593


In [39]:
# Transform power station data from NGER for loading into the dim_nger_facility table 
# Query the table and set it into a DataFrame
query = """
    SELECT *
    FROM spatial_greenhouse_and_energy
"""


# Access the tables and load into a single DataFrame
nger_df = pd.read_sql(query, con=db_engine)

# Rename the columns
nger_df = nger_df.rename(columns={
    'spatial_id': 'location_id'
})

# Now, merge cer_df with dim_fuel to get the fuel_id based on primary_fuel
nger_df = nger_df.merge(dim_fuel, how='left', left_on='primary_fuel', right_on='fuel_name')

# Drop the 'fuel_name' and 'primary_fuel' columns as it's no longer needed
nger_df = nger_df.drop(columns=['fuel_name', 'primary_fuel'])

# Reorder columns to match desired structure for dim_time
dim_nger_df = nger_df[['reporting_entity', 'facility_name', 'type', 'grid', 'fuel_id', 'location_id']]

# Ensure location_id stays null if NaN, otherwise convert to string
dim_nger_df['location_id'] = dim_nger_df['location_id'].apply(
    lambda x: str(int(x)) if pd.notna(x) and isinstance(x, (float, int)) and x.is_integer() else str(x) if pd.notna(x) else np.nan
)

# Replace NaN values in the grouping columns with 'None' as a placeholder
dim_nger_df['reporting_entity'] = dim_nger_df['reporting_entity'].fillna('None')
dim_nger_df['facility_name'] = dim_nger_df['facility_name'].fillna('None')
dim_nger_df['type'] = dim_nger_df['type'].fillna('None')
dim_nger_df['grid'] = dim_nger_df['grid'].fillna('None')
dim_nger_df['fuel_id'] = dim_nger_df['fuel_id'].fillna('None')
dim_nger_df['location_id'] = dim_nger_df['location_id'].fillna('None')

# Group by the specified columns
dim_nger_facility = dim_nger_df.groupby(
    ['reporting_entity', 'facility_name', 'type', 'grid', 'fuel_id', 'location_id']
).first().reset_index()

# Assign a unique fuel_id to each distinct fuel_name
dim_nger_facility['nger_facility_id'] = range(1, len(dim_nger_facility) + 1)

# Save a temp table to create the fact table
dim_nger_facility_copy = dim_nger_facility.copy()

# Convert 'None' placeholders back to NaN for 'fuel_id' and 'location_id'
dim_nger_facility['fuel_id'] = dim_nger_facility['fuel_id'].replace('None', np.nan).astype('Int64')
dim_nger_facility['location_id'] = dim_nger_facility['location_id'].replace('None', np.nan).astype('Int64')

# Reorder columns to match desired structure for dim_time
dim_nger_facility = dim_nger_facility[['nger_facility_id', 'reporting_entity', 'facility_name', 'type', 'grid', 'fuel_id', 'location_id']]

# Final dim_nger_facility table
dim_nger_facility

,nger_facility_id,reporting_entity,facility_name,type,grid,fuel_id,location_id
0,1,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,F,NEM,21,2316596
1,2,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,NEM,21,2316593
2,3,ACCIONA ENERGY OCEANIA PTY LTD,Mortlake South Wind Farm,F,NEM,21,2316741
3,4,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,F,NEM,21,2316741
4,5,ACCIONA ENERGY OCEANIA PTY LTD,Royalla Solar Farm,F,NEM,19,2354197
...,...,...,...,...,...,...,...
1607,1608,Water Corporation,Cocos Keeling Islands - Home Island Generation,F,Off-grid,9,2316598
1608,1609,Water Corporation,Cocos Keeling Islands - West Island Generation,F,Off-grid,9,2316598
1609,1610,YARA AUSTRALIA PTY LTD,YPF UTILITIES PRODUCTION AND SUPPLY,F,Off-grid,10,2316598
1610,1611,YARA PILBARA FERTILISERS PTY LTD,Utilities Production and Supply,F,Off-grid,10,2316598


In [40]:
# Transform the detail NGER data for loading into the fact_nger_metrics table 

# Ensure location_id stays null if NaN, otherwise convert to string
nger_df['location_id'] = nger_df['location_id'].apply(
    lambda x: str(int(x)) if pd.notna(x) and isinstance(x, (float, int)) and x.is_integer() else str(x) if pd.notna(x) else np.nan
)

# Replace NaN values in the grouping columns with 'None' as a placeholder
nger_df['reporting_entity'] = nger_df['reporting_entity'].fillna('None')
nger_df['facility_name'] = nger_df['facility_name'].fillna('None')
nger_df['type'] = nger_df['type'].fillna('None')
nger_df['grid'] = nger_df['grid'].fillna('None')
nger_df['fuel_id'] = nger_df['fuel_id'].fillna('None')
nger_df['location_id'] = nger_df['location_id'].fillna('None')

# Create composite key, skipping NaN values for each column
nger_df['composite_key'] = nger_df.apply(
    lambda row: '_'.join(
        [str(row['reporting_entity']) if pd.notna(row['reporting_entity']) else '',
         str(row['facility_name']) if pd.notna(row['facility_name']) else '',
         str(row['type']) if pd.notna(row['type']) else '',
         str(row['grid']) if pd.notna(row['grid']) else '',
         str(row['fuel_id']) if pd.notna(row['fuel_id']) else '',
         str(row['location_id']) if pd.notna(row['location_id']) else '']
    ), axis=1
)

# Clean up the composite_key by removing unnecessary leading/trailing underscores
nger_df['composite_key'] = nger_df['composite_key'].str.strip('_')

# Now, merge with dim_nger_facility to get the nger_facility_id
temp_dim_nger_facility = dim_nger_facility_copy.copy()

# Create composite key for temp_dim_nger_facility
temp_dim_nger_facility['composite_key'] = temp_dim_nger_facility.apply(
    lambda row: '_'.join(
        [str(row['reporting_entity']) if pd.notna(row['reporting_entity']) else '',
         str(row['facility_name']) if pd.notna(row['facility_name']) else '',
         str(row['type']) if pd.notna(row['type']) else '',
         str(row['grid']) if pd.notna(row['grid']) else '',
         str(row['fuel_id']) if pd.notna(row['fuel_id']) else '',
         str(row['location_id']) if pd.notna(row['location_id']) else '']
    ), axis=1
)

# Clean up the composite_key by removing unnecessary leading/trailing underscores
temp_dim_nger_facility['composite_key'] = temp_dim_nger_facility['composite_key'].str.strip('_')

# Convert 'None' placeholders back to NaN for 'fuel_id' and 'location_id'
nger_df['fuel_id'] = nger_df['fuel_id'].replace('None', np.nan).astype('Int64')
nger_df['location_id'] = nger_df['location_id'].replace('None', np.nan).astype('Int64')

# Now, merge cer_df with dim_nger_facility to get the nger_facility_id based on composite_key
nger_df = nger_df.merge(temp_dim_nger_facility[['composite_key', 'nger_facility_id']], 
                        how='left', 
                        left_on='composite_key', 
                        right_on='composite_key')

# Assign a unique nger_entry_id to each row
nger_df['nger_entry_id'] = range(1, len(nger_df) + 1)

# Create time_id (in YYYYMMDD format) for time_period_start and time_period_stop, leaving NaN as null
nger_df['time_period_start_id'] = nger_df['time_period_start'].apply(
    lambda x: x.strftime('%Y%m%d') if pd.notna(x) else np.nan
).astype('Int64')

nger_df['time_period_stop_id'] = nger_df['time_period_stop'].apply(
    lambda x: x.strftime('%Y%m%d') if pd.notna(x) else np.nan
).astype('Int64')

# Rename the columns
nger_df = nger_df.rename(columns={
    'total_scope_1_emissions_t_co2_e': 'total_co2_emissions_tons_scope_1',
    'total_scope_2_emissions_t_co2_e': 'total_co2_emissions_tons_scope_2',
    'total_emissions_t_co2_e': 'total_co2_emissions_tons'
})

# Reorder columns to match desired structure for dim_time
fact_nger_metrics = nger_df[['nger_entry_id', 'nger_facility_id', 'time_period_start_id', 'time_period_stop_id', 'electricity_production_mwh', 'electricity_production_gj', 'total_co2_emissions_tons_scope_1', 'total_co2_emissions_tons_scope_2', 'total_co2_emissions_tons', 'emission_intensity_t_co2_emwh']]

# Final dim_nger_facility table
fact_nger_metrics.head()

,nger_entry_id,nger_facility_id,time_period_start_id,time_period_stop_id,electricity_production_mwh,electricity_production_gj,total_co2_emissions_tons_scope_1,total_co2_emissions_tons_scope_2,total_co2_emissions_tons,emission_intensity_t_co2_emwh
0,1,1,20230107,20240630,133874.0,481948.0,57.0,127.0,184,0.0
1,2,2,20230107,20240630,136502.0,491409.0,50.0,218.0,268,0.0
2,3,3,20230107,20240630,283153.0,1019352.0,202.0,1128.0,1330,0.0
3,4,4,20230107,20240630,284847.0,1025451.0,99.0,1273.0,1372,0.0
4,5,6,20230107,20240630,543046.0,1954964.0,186.0,1114.0,1300,0.0


In [41]:
# Transform demographic data from abs for loading into the fact_population table 
# Query the table and set it into a DataFrame
query = """
    SELECT *
    FROM estimated_resident_population
"""

# Access the tables and load into a single DataFrame
abs_df = pd.read_sql(query, con=db_engine)

# List of columns to check
columns_to_check = [
    'year', 'estimated_resident_population_integer', 'population_density_personskm2',
    'estimated_resident_population___males_integer', 'estimated_resident_population___females_integer',
    'median_age___males_years', 'median_age___females_years', 'median_age___persons_years',
    'working_age_population_aged_15_64_years_integer', 'working_age_population_aged_15_64_years_pct'
]

# Replace 'None' with np.nan for consistency if needed
abs_df[columns_to_check] = abs_df[columns_to_check].replace('None', np.nan)

# Drop rows where all specified columns are NaN
abs_df = abs_df.dropna(subset=columns_to_check, how='all')

# Update the 'code' column where the 'label' matches a state name in the mapping
abs_df['state_code'] = abs_df.apply(
    lambda row: [key for key, value in state_name_mapping.items() if value == row['label']][0] 
    if row['label'] in state_name_mapping.values() else None,
    axis=1
)

# Rename the columns
abs_df = abs_df.rename(columns={
    'code': 'area_code',
    'estimated_resident_population_integer': 'estimated_resident_population',
    'population_density_personskm2': 'population_density_persons_km2',
    'estimated_resident_population___males_integer': 'estimated_resident_population_male',
    'estimated_resident_population___females_integer': 'estimated_resident_population_female',
    'median_age___males_years': 'median_age_male',
    'median_age___females_years': 'median_age_female',
    'median_age___persons_years': 'median_age',
    'working_age_population_aged_15_64_years_integer': 'working_age_number',
    'working_age_population_aged_15_64_years_pct': 'working_age_percentage'
})

# Assign a unique abs_entry_id to each distinct fuel_name
abs_df['abs_entry_id'] = range(1, len(abs_df) + 1)

# Reorder columns to match desired structure for dim_time
fact_population = abs_df[['abs_entry_id', 'area_code', 'state_code', 'geo_scope', 'year', 'estimated_resident_population', 'estimated_resident_population_male', 'estimated_resident_population_female', 'median_age', 'median_age_male', 'median_age_female', 'working_age_number', 'working_age_percentage']]

# Final fact_population table
fact_population.head()

,abs_entry_id,area_code,state_code,geo_scope,year,estimated_resident_population,estimated_resident_population_male,estimated_resident_population_female,median_age,median_age_male,median_age_female,working_age_number,working_age_percentage
0,1,AUS,N/A,1,2011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,AUS,N/A,1,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,AUS,N/A,1,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,AUS,N/A,1,2019.0,25334826.0,12577221.0,12757605.0,37.5,36.7,38.4,16570435.0,65.4
4,5,AUS,N/A,1,2020.0,25649248.0,12728639.0,12920609.0,37.9,37.0,38.7,16704135.0,65.1


## 6.2 Data Storage

In [43]:
try:
    # Connect to the default "postgres" database
    conn = psycopg2.connect(
        dbname="postgres", 
        user=db_user, 
        password=db_password, 
        host=db_host, 
        port=db_port
    )
    conn.autocommit = True

    cur = conn.cursor()
    # Create "energydb"
    cur.execute("CREATE DATABASE energydb;")
    cur.close()
    conn.close()

    # Now connect to "energydb" and enable PostGIS
    energy_conn = psycopg2.connect(
        dbname="energydb",
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    energy_conn.autocommit = True
    energy_cur = energy_conn.cursor()
    energy_cur.execute("CREATE EXTENSION postgis;")
    energy_cur.close()
    energy_conn.close()

except psycopg2.errors.DuplicateDatabase:
    print("Database already exists, skipping creation.")

Database already exists, skipping creation.


In [44]:
# Connect with the new energy db
energy_db_name = "energydb"
energy_db_engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{energy_db_name}')
gh_inspector = inspect(energy_db_engine)

In [45]:
with energy_db_engine.connect() as connection:
    connection.execute(text("CREATE EXTENSION IF NOT EXISTS postgis;"))

In [46]:
# Test connection and check PostGIS status using text() for SQL
with energy_db_engine.connect() as connection:
    result = connection.execute(text("SELECT PostGIS_full_version();"))
    postgis_version = result.fetchone()
    print(f"PostGIS Version: {postgis_version[0]}")

PostGIS Version: POSTGIS="3.6.0 0" [EXTENSION] PGSQL="140" GEOS="3.14.0-CAPI-1.20.4" PROJ="9.6.2 NETWORK_ENABLED=OFF URL_ENDPOINT=https://cdn.proj.org USER_WRITABLE_DIRECTORY=/Users/naufal/Library/Application Support/proj DATABASE_PATH=/opt/homebrew/Cellar/proj/9.6.2/share/proj/proj.db" (compiled against PROJ 9.6.2) LIBXML="2.13.8" LIBJSON="0.18" LIBPROTOBUF="1.5.2" WAGYU="0.5.0 (Internal)"


In [47]:
# Drop tables in the correct dependency order
drop_tables_sql = """
DROP TABLE IF EXISTS 
    fact_nger_metrics,
    fact_population,
    dim_nger_facility,
    dim_cer_station,
    dim_location,
    dim_state,
    dim_fuel,
    dim_time
CASCADE;
"""

# Create the tables for our schema

# Dimension Tables

dim_fuel_sql = """
CREATE TABLE IF NOT EXISTS dim_fuel (
    fuel_id SERIAL PRIMARY KEY,
    fuel_name TEXT
);
"""

dim_cer_station_sql = """
CREATE TABLE IF NOT EXISTS dim_cer_station (
    cer_station_code BIGSERIAL PRIMARY KEY,
    accreditation_code TEXT,
    station_name TEXT,
    accreditation_start_date_id BIGINT,
    committed_date_id BIGINT,
    status TEXT,
    baseline_mwh FLOAT,
    installed_capacity FLOAT,
    suspension_status TEXT,
    fuel_id BIGINT,
    location_id BIGINT,
    FOREIGN KEY (fuel_id) REFERENCES dim_fuel(fuel_id)
);
"""

dim_state_sql = """
CREATE TABLE IF NOT EXISTS dim_state (
    state_code TEXT PRIMARY KEY,
    state_name TEXT
);
"""

dim_location_sql = """
CREATE TABLE IF NOT EXISTS dim_location (
    location_id BIGSERIAL PRIMARY KEY,
    latitude FLOAT,
    longitude FLOAT,
    geometry GEOMETRY(POINT, 4326),
    postcode TEXT, -- Nullable
    state_code TEXT, -- Foreign Key to dim_state
    FOREIGN KEY (state_code) REFERENCES dim_state(state_code)
);
"""

dim_nger_facility_sql = """
CREATE TABLE IF NOT EXISTS dim_nger_facility (
    nger_facility_id SERIAL PRIMARY KEY,
    reporting_entity TEXT,
    facility_name TEXT,
    type TEXT,
    grid TEXT,
    fuel_id INT, -- A foreign key to dim_fuel
    location_id BIGINT, -- A foreign key to dim_location
    FOREIGN KEY (fuel_id) REFERENCES dim_fuel(fuel_id),
    FOREIGN KEY (location_id) REFERENCES dim_location(location_id)
);
"""

dim_time_sql = """
CREATE TABLE IF NOT EXISTS dim_time (
    time_id INT PRIMARY KEY,
    date DATE,
    day INT,
    month INT,
    month_name TEXT,
    quarter INT,
    year INT,
    week_of_year INT,
    day_of_week INT,
    day_name TEXT,
    is_weekend BOOLEAN
);
"""

# Fact Tables

fact_nger_metrics_sql = """
CREATE TABLE IF NOT EXISTS fact_nger_metrics (
    nger_entry_id SERIAL PRIMARY KEY,
    nger_facility_id INT, -- Foreign Key to dim_nger_facility
    time_period_start_id INT, -- Foreign Key to dim_time
    time_period_stop_id INT, -- Foreign Key to dim_time
    electricity_production_mwh FLOAT,
    electricity_production_gj FLOAT,
    total_emissions_tons FLOAT,
    total_co2_emissions_tons_scope_1 FLOAT,
    total_co2_emissions_tons_scope_2 FLOAT,
    total_co2_emissions_tons FLOAT,
    emission_intensity_t_co2_emwh FLOAT,
    FOREIGN KEY (nger_facility_id) REFERENCES dim_nger_facility(nger_facility_id),
    FOREIGN KEY (time_period_start_id) REFERENCES dim_time(time_id),
    FOREIGN KEY (time_period_stop_id) REFERENCES dim_time(time_id)
);
"""

fact_population_sql = """
CREATE TABLE IF NOT EXISTS fact_population (
    abs_entry_id SERIAL PRIMARY KEY,
    area_code TEXT, 
    state_code TEXT, -- Foreign Key to dim_state
    geo_scope INT,
    year INT,
    estimated_resident_population INT,
    estimated_resident_population_male INT,
    estimated_resident_population_female INT,
    population_density_persons_km2 FLOAT,
    median_age FLOAT,
    median_age_male FLOAT,
    median_age_female FLOAT,
    working_age_number INT,
    working_age_percentage FLOAT,
    FOREIGN KEY (state_code) REFERENCES dim_state(state_code)
);
"""

# Execute the SQL to create tables
with energy_db_engine.begin() as connection:
    try:
        # Drop all tables first
        connection.execute(text(drop_tables_sql))

        # Create fresh schema
        connection.execute(text(dim_fuel_sql))
        connection.execute(text(dim_cer_station_sql))
        connection.execute(text(dim_state_sql))
        connection.execute(text(dim_location_sql))
        connection.execute(text(dim_nger_facility_sql))
        connection.execute(text(dim_time_sql))
        connection.execute(text(fact_nger_metrics_sql))
        connection.execute(text(fact_population_sql))
    except Exception as e:
        print("Error during table creation:", e)

In [48]:
# Safe truncate of tables while preserving schema
with energy_db_engine.begin() as conn:  # `begin()` ensures transactional integrity
    conn.execute(text("TRUNCATE TABLE dim_fuel CASCADE"))
    conn.execute(text("TRUNCATE TABLE dim_cer_station CASCADE"))
    conn.execute(text("TRUNCATE TABLE dim_state CASCADE"))
    conn.execute(text("TRUNCATE TABLE dim_location CASCADE"))
    conn.execute(text("TRUNCATE TABLE dim_nger_facility CASCADE"))
    conn.execute(text("TRUNCATE TABLE dim_time CASCADE"))
    conn.execute(text("TRUNCATE TABLE fact_nger_metrics CASCADE"))
    conn.execute(text("TRUNCATE TABLE fact_population CASCADE"))

# Now insert the data into the tables
dim_fuel.to_sql('dim_fuel', con=energy_db_engine, if_exists='append', index=False)
dim_cer_station.to_sql('dim_cer_station', con=energy_db_engine, if_exists='append', index=False)
dim_states.to_sql('dim_state', con=energy_db_engine, if_exists='append', index=False)
dim_location.to_sql('dim_location', con=energy_db_engine, if_exists='append', index=False)
dim_nger_facility.to_sql('dim_nger_facility', con=energy_db_engine, if_exists='append', index=False)
dim_time.to_sql('dim_time', con=energy_db_engine, if_exists='append', index=False)
fact_nger_metrics.to_sql('fact_nger_metrics', con=energy_db_engine, if_exists='append', index=False)
fact_population.to_sql('fact_population', con=energy_db_engine, if_exists='append', index=False)

print("Data inserted successfully.")

Data inserted successfully.
